### Initial Setup

 - initialize the repositories and MongoDB indexes [see commands here](../ToolBox/Repository-Cleanup.ipynb)
    - for `us-east` and `us-west`

### Import in US-EAST

In [2]:
# import in synchronous mode
!import.sh -o import -l import/states-hierarchy-us-east -r us-east -b /

Using config /nxbench/notebooks/11B-Steps/nuxeo.properties
url=http://127.0.0.1:8080/nuxeo
login=nco-admin
Nuxeo Client configured
Connected to Nuxeo Server 11.3.26
Running Operation:StreamImporter.runDocumentConsumersEx
   nbThreads: 10 
   logName: import/states-hierarchy-us-east 
   blockDefaultSyncListeners: true 
   rootFolder: / 
   logSize: 8 
   batchSize: 500 
#####################
Execution completed
elapsed:60.047
committed:26
failures:0
consumers:8
throughput:0.4329941545789132



In [3]:
# import customers
nbDocs = 89997827
expectedThroughput = 14000
print("expected duration (s) =", round(nbDocs/expectedThroughput))
print("expected duration (h) =", round((nbDocs/expectedThroughput)/3600))

expected duration (s) = 6428
expected duration (h) = 2


In [ ]:
!import.sh -o import -t 16 -l import/customers-us-east -r us-east -b / -a -w 8000 -bulk > import-useast-customers.log

In [5]:
!tail import-useast-customers.log

Nuxeo Client configured
.........................................................................................................................
Running completed
elapsed:7301.021
committed:90918788
failures:0
consumers:16
throughput:12452.886794874306

Exit after 7323 s


In [8]:
# import accounts
nbDocs = 89997827*2
expectedThroughput = 10000
print("expected duration (s) =", round(nbDocs/expectedThroughput))
print("expected duration (h) =", round((nbDocs/expectedThroughput)/3600))

expected duration (s) = 18000
expected duration (h) = 5


In [ ]:
!import.sh -o import -t 16 -l import/accounts-us-east -r us-east -b / -a -w 25000 -bulk > import-useast-accounts.log

In [10]:
# import statements
nbDocs = 89997827*6
expectedThroughput = 10000
print("expected duration (s) =", round(nbDocs/expectedThroughput))
print("expected duration (h) =", round((nbDocs/expectedThroughput)/3600))

expected duration (s) = 53999
expected duration (h) = 15


In [ ]:
!import.sh -o import -t 16 -l import/statements_archive0-us-east -r us-east -b / -a -w 60000 -bulk

### Indexing US-EAST

 - scale out the number of Nuxeo Worker nodes
 - configure ES for bulk indexing (see [ES toolbox](../ToolBox/Elasticsearch.ipynb))
     - no replicas
     - refresh rate
     
 Start Indexing on US-EAST using BAF    

In [27]:
!(INJECTOR="http://127.0.0.1:8080";\
curl -H 'Content-Type:application/json+nxrequest' \
  -H 'X-NXRepository:us-east' \
  -X POST -d '{"params":{},"context":{}}' -u $NXUSER:$NXPWD \
  "$INJECTOR/nuxeo/api/v1/automation/Elasticsearch.BulkIndex")

curl: (7) Failed to connect to 127.0.0.1 port 8080: Connection refused


In [29]:
# check status
! cid="6057b8cb-c073-466c-96c5-9252aee168cc"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:us-east' \
  -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid

curl: (7) Failed to connect to 127.0.0.1 port 8080: Connection refused


### Import in US-WEST

In [ ]:
# import in synchronous mode
!import.sh -o import -l import/states-hierarchy-us-west -r us-west -b /

In [ ]:
!import.sh -o import -t 16 -l import/customers-us-west -r us-west -b / -a -w 8000 -bulk

In [ ]:
!import.sh -o import -t 16 -l import/accounts-us-west -r us-west -b / -a -w 20000 -bulk

In [ ]:
!import.sh -o import -t 16 -l import/statements_archive0-us-west -r us-west -b / -a -w 60000 -bulk

### Indexing US-WEST
 
 - configure ES for bulk indexing (see [ES toolbox](../ToolBox/Elasticsearch.ipynb))
     - no replicas
     - refresh rate
     
Start Indexing on US-WEST using BAF    

In [ ]:
!(INJECTOR="http://127.0.0.1:8080";\
curl -H 'Content-Type:application/json+nxrequest' \
  -H 'X-NXRepository:us-west' \
  -X POST -d '{"params":{},"context":{}}' -u $NXUSER:$NXPWD \
  "$INJECTOR/nuxeo/api/v1/automation/Elasticsearch.BulkIndex")

In [ ]:
# check status
! cid="6057b8cb-c073-466c-96c5-9252aee168cc"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:us-west' \
  -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid